In [28]:
# try:
#     get_ipython().system('pip install pandas_datareader')
#     get_ipython().system('pip install pandas')
#     get_ipython().system('pip install matplotlib')
#     get_ipython().system('pip install requests')
#     get_ipython().system('pip install sympy')
#     get_ipython().system('pip install scipy')
#     get_ipython().system('pip install numpy')
#     get_ipython().system('pip install beautifulsoup4')
#     get_ipython().system('pip install numpy-financial')
# except Exception:
#     pass

In [29]:
import requests
import numpy_financial as npf
import numpy as np
from bs4 import BeautifulSoup
import threading
import re
import math

In [30]:
tickers = []

with open('tickers.csv') as fh:
    for line in fh:
        row = line.strip()
        tickers.append(row)

In [31]:
def rdpModelReturn(payoutRatio, PE, returnOnEquity):
    return (1 + returnOnEquity*(1-payoutRatio))*(1+payoutRatio/PE)-1

In [32]:
# jitta temp mail: fesodiha@clout.wiki
# copy this following code to extract cookies from jitta.com and then manual replace number to string for some key, remove nested field
# document.cookie.split(';').map((c) => c.trim().split('=').map(decodeURIComponent)).reduce((a, b) => {try {a[b[0]] = JSON.parse(b[1]);}catch(e) {a[b[0]] = b[1];}return a;}, {});

cookies = {
    "_gid": "GA1.2.1787214795.1680600058",
    "ajs_anonymous_id": "ba83fd06-acd4-491c-87f0-93d5a0ec1167",
    "_fbp": "fb.1.1680600058107.1290303553",
    "gclid": "undefined",
    "hunt7586-_zldp": "ss+YrC93NjOVwgyi/EREBDDyggBwhhrsjOe1mSQ5qB4Gl/cFmC1zp9ADOaZpPlyn7uRfjfIQJjw=",
    "hunt7586-_zldt": "cffe8506-b037-4727-b0fe-0df85ace2121-2",
    "JDCID": "efbbd5c32d34555ef7b4bbcbc9a0af863da8f46f65c27f7c67b4e5e9a5b97cfa.5439c914472955e6ecc4ac72cbf4f297dafa500b94dc38a5e2b63278cc6c5369275321157f5bbd49210a4b2cc4dec3851703061b2613e31a45162e61f84910a362b436eb4058a50cb920a0f8f433b0620c295848890c",
    "_ga": "GA1.1.1731066012.1680600058",
    "_ga_BT0RBD348J": "GS1.1.1680600057.1.1.1680600136.44.0.0",
    "ajs_user_id": "642bec1b2e042000174a3d46",
    "jitta_nu": "2"
}

In [33]:
def convert_row_to_data(row):
    r = row.find_all('span')
    c = map(lambda x: p2f(x.text), r)
    return list(c)[:-2]

def p2f(x):
    cx = x.replace('%', '').replace(',', '')
    return None if cx == '- -' or cx == '?' else float(cx)

def rate_of_change(nums):
    return list(map(lambda i: None if nums[i-1] is None or nums[i] is None else nums[i] / nums[i-1] - 1, range(1, len(nums))))

def mean(nums):
    nums = list(filter(lambda x: x is not None, nums))
    return 0.0 if len(nums) == 0 else sum(nums) / len(nums)

def median(nums):
    nums = sorted(filter(lambda x: x is not None, nums))
    n = len(nums)
    if n == 0:
        return 0.0
    if n % 2 == 1:
        return nums[n // 2]
    else:
        return (nums[n // 2 - 1] + nums[n // 2]) / 2


def clean_data(y):
    n = len(y)
    if n == 0:
        return []
    cleaned_y = [0.0] * n
    cleaned_y[0] = y[0] if y[0] is not None else 0.0
    cleaned_y[-1] = y[-1] if y[-1] is not None else 0.0
    for i in range(1, n-1):
        if y[i] is None:
            cleaned_y[i] = 0.0
        elif abs(y[i] - cleaned_y[i-1]) > abs(y[i] - y[i+1]):
            cleaned_y[i] = y[i+1]
        else:
            cleaned_y[i] = y[i-1]
    return cleaned_y


def linear_regression(y):
    cleaned_y = clean_data(y)
    n = len(cleaned_y)
    x = list(range(n))
    x_mean = sum(x) / n
    y_mean = sum(cleaned_y) / n
    cov = sum([(x[i] - x_mean) * (cleaned_y[i] - y_mean) for i in range(n)])
    var = sum([(x[i] - x_mean) ** 2 for i in range(n)])
    slope = cov / var
    intercept = y_mean - slope * x_mean
    y_pred = [slope * i + intercept for i in x]
    roc = 0 if y_mean == 0 else slope / y_mean
    return x, slope, intercept, y_pred, roc, y_mean

def dcf_irr(price, eps, g, dpr):
    tax_rate = 0.15
    adj_dpr = (1 + dpr * (1 - tax_rate))
    cashflows = [price * -1]
    g1 = g
    g2 = 0.5 * g
    g3 = min(0.10, 0.2 * g)
    g4 = min(0.04, 0.1 * g)
    last_eps = eps
    
    for i in range(0, 40):
        if i < 5:
            last_eps = last_eps * (1 + g1)
        elif i < 10:
            last_eps = last_eps * (1 + g2)
        elif i < 20:
            last_eps = last_eps * (1 + g3)
        else:
            last_eps = last_eps * (1 + g4)

        cashflows.append(last_eps * adj_dpr)
    irr = npf.irr(cashflows)
    return 0.0 if np.isnan(irr) else irr

In [34]:
def calculate(ticker):
    with requests.post('https://www.jitta.com/stock/' + ticker + '/factsheet', cookies=cookies) as r:
        soup = BeautifulSoup(r.content, 'lxml')

        try:
            price = float(re.search(
                r"\.price\.latest\":{\"datetime\":\"(.*)\",\"close\":(.*),\"market_cap\"", r.content.decode('utf-8'))[2])

            row = soup.find_all('div', {'class': 'dAGwCX'})

            if len(row) == 0:
                return None
        except:
            return None

        REV = convert_row_to_data(row[0])
        EPS = convert_row_to_data(row[1])
        ROE = convert_row_to_data(row[20])
        ROA = convert_row_to_data(row[21])
        DE = convert_row_to_data(row[23])
        DPR = convert_row_to_data(row[24])

        eps_forecast = linear_regression(
            list(map(lambda x: 0 if x == None else x, EPS)))
        dpr_forecast = linear_regression(
            list(map(lambda x: 0 if x == None else max(0, min(100, x)), DPR)))
        print(DPR)
        print(dpr_forecast)

        eps = float(eps_forecast[3][-1])
        eps_pos = len(list(filter(lambda x: x is not None and x > 0, EPS)))
        g = float(eps_forecast[4])
        dpr = float(dpr_forecast[3][-1] / 100)

        irr = float(dcf_irr(price, eps, g, dpr))

        print(f"{ticker} {irr:.2} {g:.2} {eps:.2} {dpr:.2}")

        return {
            "TICKER": ticker,
            "PRICE": price,
            "EPS": eps,
            "EPS_POS": eps_pos,
            "EPS_N": len(EPS),
            "G": g,
            "DPR": dpr,
            "IRR": irr,
        }


In [35]:
f = open('stock-screen.csv', 'w')
f.write('ticker|IRR|G|EPS|EPS_POS|EPS_N|DPR|PRICE\n')


def fetch_stock(ticker):
    data = calculate(ticker)

    if data == None:
        return

    f.write(ticker + '|' +
            '{:.4f}'.format(data['IRR']) + '|' +
            '{:.4f}'.format(data['G']) + '|' +
            '{:.4f}'.format(data['EPS']) + '|' +
            '{:.4f}'.format(data['EPS_POS']) + '|' +
            '{:.4f}'.format(data['EPS_N']) + '|' +
            '{:.4f}'.format(data['DPR']) + '|' +
            '{:.4f}'.format(data['PRICE']) + '\n'
            )


In [36]:
threads  = []
for ticker in tickers:
    fetch_stock(ticker)

f.close()

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, None]
([0, 1, 2, 3, 4, 5, 6, 7, 8], 0.0, 0.0, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 0, 0.0, 60.0)
nasdaq:AACG 0.0 0.23 -0.43 0.0
[None, None, None, None, None, None, None, None, None]
([0, 1, 2, 3, 4, 5, 6, 7, 8], 0.0, 0.0, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 0, 0.0, 60.0)
nasdaq:AACI 0.0 0.6 -0.064 0.0
[None, None, None, None, None, None, 0.0, None, None]
([0, 1, 2, 3, 4, 5, 6, 7, 8], 0.0, 0.0, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 0, 0.0, 60.0)
nasdaq:AADI 0.0 0.53 -1.9 0.0
[5.1, 3.6, 8.6, 15.3, 13.2, 10.6, -0.5, None, None]
([0, 1, 2, 3, 4, 5, 6, 7, 8], -0.7149999999999999, 9.026666666666667, [9.026666666666667, 8.311666666666667, 7.596666666666668, 6.881666666666668, 6.166666666666668, 5.451666666666668, 4.736666666666668, 4.021666666666668, 3.3066666666666684], -0.11594594594594591, 6.166666666666667, 60.0)
nasdaq:AAL 0.072 -2.4 -6.7 0.033
[10.5, 10.5, 18.3, 10.0, -36.8, None, 3.6, 10.5, None]
([0, 1, 2, 3, 4, 5

KeyboardInterrupt: 